In [2]:
import pandas as pd
import json

def load_seal_json():
    with open('seal_para_data.json') as f:
        seal_para_data = json.load(f)
        return seal_para_data
# load_seal_json() # success

def json2excel():
    data = load_seal_json()
    # 创建一个DataFrame
    df = pd.DataFrame(data)
    # 将DataFrame写入Excel文件
    df.to_excel('seal_attribute_raw.xlsx', index=False)  # 如果你不想写入行索引，请使用index=False
    
# json2excel()

In [4]:
import pandas as pd
import json
# 设置相对路径，将路径定位为当前文件夹
import os  # 导入os模块
os.chdir('E:\\DataVis\\vis_system\\seal_visualization\\data_process') # 更改当前工作目录

def xlsx2json():
    # 用 Pandas 读取.xlsx文件
    data = pd.read_excel('seal_attribute.xlsx')
    print(data)
    # 将 Pandas 数据框转换为 JSON
    json_data = data.to_dict(orient='records')
    print(json_data)
    
    collector_dict = getCollectorJson(json_data)
    for seal in json_data:
        json_data[json_data.index(seal)]['seal_owner_id'] = collector_dict[seal['seal_owner']]

    # 将 JSON 数据保存到文件
    with open('./seal_data/seal_data.json', 'w') as json_file:
        json.dump(json_data, json_file, indent=4, ensure_ascii=False)
        
def getCollectorJson(seal_list):
    collector_list = []
    for seal in seal_list:
        collector_list.append(seal['seal_owner'])
    collector_list = list(set(collector_list))
    for collector in collector_list:
        collector_list[collector_list.index(collector)] = {
            'collector_name': collector,
            'index': str(collector_list.index(collector))
        }
    collector_dict = {}
    for collector in collector_list:
        collector_dict[collector['collector_name']] = collector['index'] # name: index(id)
    with open('./seal_data/seal_owner.json', 'w') as json_file:
        json.dump(collector_dict, json_file, indent=4, ensure_ascii=False)
    return collector_dict
xlsx2json()

     index   seal_name seal_owner      x  x_right     y  width  height
0        0  五福五代堂古稀天子寶    爱新觉罗·弘历  19685    45815     0    155     268
1        1        乾隆宸翰    爱新觉罗·弘历  49957    15543     2    123     282
2        2          --         --  20369    45131     2    218     327
3        3        蕉林書屋        梁清标  38243    27257     5    167     289
4        4          --         --  48917    16583    26    437     539
..     ...         ...        ...    ...      ...   ...    ...     ...
185    185        花间草堂       纳兰性德  31200    34300  3238    323     330
186    186        冶溪漁隱        梁清标  20305    45195  3302    283     274
187    187        曹溶之印         曹溶  20624    44876  3327    238     242
188    188          鉴躬         曹溶  19621    45879  3335    244     239
189    189         芝石室         --   8052    57448  3228    224     226

[190 rows x 8 columns]
[{'index': 0, 'seal_name': '五福五代堂古稀天子寶', 'seal_owner': '爱新觉罗·弘历', 'x': 19685, 'x_right': 45815, 'y': 0, 'width': 155, 'heigh

In [3]:
import pandas as pd
import json
# 枚举印章名称(for test)
def enumerate_seal_names():
    enumerate_seal_list = []
    # 用 Pandas 读取.xlsx文件
    data = pd.read_excel('seal_attribute.xlsx')
    print(data)
    # 将 Pandas 数据框转换为 JSON
    json_data = data.to_dict(orient='records')
    for seal in json_data:
        enumerate_seal_list.append(seal['seal_name'])
    enumerate_seal_list = sorted(list(set(enumerate_seal_list)))
    with open('./seal_data/seal_set.json', 'w') as json_file:
        json.dump(enumerate_seal_list, json_file, indent=4, ensure_ascii=False)
# enumerate_seal_names()

     index   seal_name seal_owner      x  x_right     y  width  height
0        0  五福五代堂古稀天子寶    爱新觉罗·弘历  19685    45815     0    155     268
1        1        乾隆宸翰    爱新觉罗·弘历  49957    15543     2    123     282
2        2          --         --  20369    45131     2    218     327
3        3        蕉林書屋        梁清标  38243    27257     5    167     289
4        4          --         --  48917    16583    26    437     539
..     ...         ...        ...    ...      ...   ...    ...     ...
185    185        花间草堂       纳兰性德  31200    34300  3238    323     330
186    186        冶溪漁隱        梁清标  20305    45195  3302    283     274
187    187        曹溶之印         曹溶  20624    44876  3327    238     242
188    188          鉴躬         曹溶  19621    45879  3335    244     239
189    189         芝石室         --   8052    57448  3228    224     226

[190 rows x 8 columns]


In [19]:
# 将数据转换为前端需要的数据结构
def trans_data_structure():
    output_data = []
    seal_pic_data, seal_owner_data = [], {}
    # 读取印章图片的文件
    with open('./seal_data/seal_data.json', "r") as json_file:
        seal_pic_data = json.load(json_file)
    # 读取人物
    with open('./seal_data/seal_owner.json', "r") as json_file:
        seal_owner_data = json.load(json_file)
    for collector in seal_owner_data:
        output_data.append({
            "collector_id": 'owner-' + seal_owner_data[collector], # 'owner-0'的形式
            "collector_id_ori": seal_owner_data[collector], # 原id，与印章图片的id重复，查找完成后待删除
            "collector_name": collector,
            "life_span": '', # 待添加
            "intro": '', # 待添加
            "seals": []
        })
    # print(output_data)
    for seal_pic in seal_pic_data:
        for collector in output_data:
            if collector['collector_id_ori'] == seal_pic['seal_owner_id']: # 印章图片属于该鉴藏者
                find_seal_name = False
                for seal in collector['seals']:
                    # print('seal', seal)
                    if seal['seal_name'] == seal_pic['seal_name']: # 已经有该印章名称
                        find_seal_name = True
                        output_data[output_data.index(collector)]['seals'][collector['seals'].index(seal)]['seal_pic'].append({
                            "index": str(seal_pic['index']),
                            "x": seal_pic['x'],
                            "x_right": seal_pic['x_right'],
                            "y": seal_pic['y'],
                            "width": seal_pic['width'],
                            "height": seal_pic['height']
                        })
                        print(collector['seals'])
                        break
                    else:
                        continue
                if not find_seal_name:
                    if 'seals' in collector:
                        output_data[output_data.index(collector)]['seals'].append({
                            "seal_name": seal_pic['seal_name'],
                            "seal_name_id": 'seal_name-' + str(len(collector['seals'])),
                            "seal_pic": [
                                {
                                    "index": str(seal_pic['index']),
                                    "x": seal_pic['x'],
                                    "x_right": seal_pic['x_right'],
                                    "y": seal_pic['y'],
                                    "width": seal_pic['width'],
                                    "height": seal_pic['height'] 
                                }
                            ]
                        })
                    else:
                        output_data[output_data.index(collector)]['seals'] = [
                            {
                                "seal_name": seal_pic['seal_name'],
                                "seal_name_id": 'seal_name-' + str(len(collector['seals'])),
                                "seal_pic": [
                                    {
                                        "index": str(seal_pic['index']),
                                        "x": seal_pic['x'],
                                        "x_right": seal_pic['x_right'],
                                        "y": seal_pic['y'],
                                        "width": seal_pic['width'],
                                        "height": seal_pic['height'] 
                                    }
                                ]
                            }
                        ]
                    output_data[output_data.index(collector)] = collector
            else:
                continue
    output_data = [
        {
            "painting_name": "鹊华秋色图卷",
            "meta_data": '', # 待补充
            "collectors": output_data
        }
    ]
    with open('./seal_data/constructed_data.json', 'w') as json_file:
        json.dump(output_data, json_file, indent=4, ensure_ascii=False)
trans_data_structure()

[{'seal_name': '--', 'seal_name_id': 'seal_name-0', 'seal_pic': [{'index': '2', 'x': 20369, 'x_right': 45131, 'y': 2, 'width': 218, 'height': 327}, {'index': '4', 'x': 48917, 'x_right': 16583, 'y': 26, 'width': 437, 'height': 539}]}]
[{'seal_name': '--', 'seal_name_id': 'seal_name-0', 'seal_pic': [{'index': '2', 'x': 20369, 'x_right': 45131, 'y': 2, 'width': 218, 'height': 327}, {'index': '4', 'x': 48917, 'x_right': 16583, 'y': 26, 'width': 437, 'height': 539}, {'index': '6', 'x': 49423, 'x_right': 16077, 'y': 35, 'width': 242, 'height': 255}]}]
[{'seal_name': '--', 'seal_name_id': 'seal_name-0', 'seal_pic': [{'index': '2', 'x': 20369, 'x_right': 45131, 'y': 2, 'width': 218, 'height': 327}, {'index': '4', 'x': 48917, 'x_right': 16583, 'y': 26, 'width': 437, 'height': 539}, {'index': '6', 'x': 49423, 'x_right': 16077, 'y': 35, 'width': 242, 'height': 255}, {'index': '14', 'x': 49745, 'x_right': 15755, 'y': 94, 'width': 152, 'height': 161}]}, {'seal_name': '长庆', 'seal_name_id': 'seal_nam

In [20]:
import shutil
# 后续手动添加了内容，不可覆盖
# 拷贝到 data 下
# shutil.copy('./seal_data/constructed_data.json', 'E:/DataVis/vis_system/seal_visualization/web/public/data/seal_constructed_data.json')

'E:/DataVis/vis_system/seal_visualization/web/public/data/seal_constructed_data.json'